In [ ]:
import os
import cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#압축파일 풀기
%cd /content
!mkdir Seg
%cd /content/Seg
!unzip -qq "/content/drive/MyDrive/Surface_1.zip"

/content
/content/Seg


In [ ]:
#폴더 읽어서 이미지 경로, xml 경로 저장
from glob import glob

folder_path = glob('/content/Seg/*')
print(len(folder_path))

from os import listdir

def fileids(path, ext = None):
    fileList = list()
    path = path if path[-1] == '/' else path + '/'
    for fileName in listdir(path):
        if fileName.endswith(ext):
            fileList.append(path + fileName)
    return fileList


image_list_list = [fileids(path, ext = 'jpg') for path in folder_path]

image_list = list()
for _ in image_list_list:
  image_list += _

xml_path = [fileids(path, ext = 'xml') for path in folder_path]

#마스크 경로도 저장
mask_list = glob('/content/drive/MyDrive/Mask/*')

130


In [ ]:
#마스크 이미지 없을 경우 실행

#imgdict 만들기
from bs4 import BeautifulSoup
from tqdm import tqdm

#filepath 코드로 xml_path 뽑아서 진행

imgdict = dict()

for xml in tqdm(xml_path):
    xml = BeautifulSoup(open(xml[0], encoding='utf-8'),'xml')
    for image in xml.select('image'):
        name = image['name']
        imgdict[name] = {
            'height': image['height'],
            'width' : image['width'],
            'polygons' : list()
        }
        for polygon in image.select('polygon'):
            label = polygon['label']
            attr = polygon.text.strip('\n')
            points = polygon['points']
            z_order = polygon['z_order']
            imgdict[name]['polygons'].append([label+'_'+attr, points, z_order])
print(len(imgdict))

#labeldict 만들기
labels = list()
for image in imgdict:
    for polygon in imgdict[image]['polygons']:
        labels.append((polygon[0]))

print(len(labels))

labels = set(labels)

print(len(labels))

labels = list(labels)

labels.sort()

labeldict = dict()

for i,label in enumerate(labels):
    labeldict[label] = i
  
# 마스킹 이미지 뽑기, 마스크 리스트도 만들기

import cv2 as cv
import numpy as np
from PIL import Image
from tqdm import tqdm

mask_list = list()

for image in tqdm(imgdict):

    height = int(imgdict[image]['height'])
    width = int(imgdict[image]['width'])
    
    mask = np.zeros((height, width,3), np.uint8)
    
    for polygon in imgdict[image]['polygons']:
        label = polygon[0]
        points = np.array([[float(p.split(',')[0]),float(p.split(',')[1])] for p in polygon[1].split(';')])
        points = np.rint(points - np.floor(points/points.max(axis=0))).astype(int)
        if label in labeldict:
            mask = cv.fillPoly(mask,[points],(labeldict[label],0,0))
        else:
            mask = cv.fillPoly(mask,[points],(0,0,0))
    
    mask = Image.fromarray(mask[...,0])
    mask.save('/content/drive/MyDrive/Mask/' + image[:-4] + '.png', 'PNG')
    mask_list.append('/content/drive/MyDrive/Mask/' + image[:-4] + '.png')


100%|██████████| 130/130 [00:04<00:00, 30.23it/s]


9558
40269
22


100%|██████████| 9558/9558 [07:07<00:00, 22.35it/s]


In [ ]:
IMAGE_SIZE = 512
BATCH_SIZE = 4
NUM_CLASSES = 22
NUM_TRAIN_IMAGES = 9000
NUM_VAL_IMAGES = 558

# 이미지 경로 수정 필요
train_images = sorted(image_list)[:NUM_TRAIN_IMAGES]
train_masks = sorted(mask_list)[:NUM_TRAIN_IMAGES]
val_images = sorted(image_list)[NUM_TRAIN_IMAGES : (NUM_VAL_IMAGES + NUM_TRAIN_IMAGES)]
val_masks = sorted(mask_list)[NUM_TRAIN_IMAGES : (NUM_VAL_IMAGES + NUM_TRAIN_IMAGES)]

# 마스크 이미지를 그대로 써보기 위해 사용
def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.image.decode_png(image, channels=1)
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    else:
        image = tf.image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = image / 127.5 - 1
    return image


def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask


def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset


train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

Train Dataset: <BatchDataset element_spec=(TensorSpec(shape=(4, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 512, 512, 1), dtype=tf.float32, name=None))>
Val Dataset: <BatchDataset element_spec=(TensorSpec(shape=(4, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 512, 512, 1), dtype=tf.float32, name=None))>


In [ ]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output


In [ ]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights ="imagenet", include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [ ]:
# model.load_weights('/content/drive/MyDrive/Deeplab_callbacks/01-1.39.hdf5')

In [ ]:
my_callbacks = [
    keras.callbacks.CSVLogger('/content/drive/MyDrive/Deeplablog/log.csv', separator=",", append=True),
    keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Deeplablog/Weights/{epoch:02d}-{val_loss:.2f}.hdf5', 
                                    monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=True, mode='auto', save_freq='epoch')
]

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=loss,
    metrics=["accuracy"],
)

history = model.fit(train_dataset, validation_data=val_dataset, epochs=100, callbacks=my_callbacks) # epoch 설정!

plt.plot(history.history["loss"])
plt.title("Training Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["accuracy"])
plt.title("Training Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_loss"])
plt.title("Validation Loss")
plt.ylabel("val_loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_accuracy"])
plt.title("Validation Accuracy")
plt.ylabel("val_accuracy")
plt.xlabel("epoch")
plt.show()

while True:
  pass

Epoch 1/100
2250/2250 [==============================] - 902s 399ms/step - loss: 1.0758 - accuracy: 0.6870 - val_loss: 0.8719 - val_accuracy: 0.7710
Epoch 2/100
2250/2250 [==============================] - 897s 399ms/step - loss: 0.9895 - accuracy: 0.7060 - val_loss: 0.8310 - val_accuracy: 0.7810
Epoch 3/100
2250/2250 [==============================] - 898s 399ms/step - loss: 0.9214 - accuracy: 0.7240 - val_loss: 0.7687 - val_accuracy: 0.7928
Epoch 4/100
2250/2250 [==============================] - 898s 399ms/step - loss: 0.8755 - accuracy: 0.7352 - val_loss: 0.7590 - val_accuracy: 0.7986
Epoch 5/100
2250/2250 [==============================] - 896s 398ms/step - loss: 0.8376 - accuracy: 0.7454 - val_loss: 0.7444 - val_accuracy: 0.8011
Epoch 6/100
2250/2250 [==============================] - 897s 399ms/step - loss: 0.7979 - accuracy: 0.7556 - val_loss: 0.7351 - val_accuracy: 0.8036
Epoch 7/100
2250/2250 [==============================] - 896s 398ms/step - loss: 0.7708 - accuracy: 0.7623

In [ ]:
# Loading the Colormap
colormap = np.array([[ 0,  0,  0],
       [50,  0,  0],
       [99,  0,  0],
       [ 0, 33,  0],
       [66,  0, 19],
       [99, 33,  0],
       [ 0,  0, 33],
       [ 0, 46, 86],
       [33, 33,  0],
       [ 0, 33, 33],
       [33, 19,  0],
       [20, 33, 50],
       [ 0, 50,  0],
       [ 0,  0, 99],
       [19, 66, 86],
       [ 0, 99, 99],
       [33, 99, 66],
       [66, 99, 33],
       [99, 99,  0],
       [99, 66,  0],
       [0, 50, 99]])
       
colormap = colormap.astype(np.uint8)


def infer(model, image_tensor):
    predictions = model.predict(np.expand_dims((image_tensor), axis=0))
    predictions = np.squeeze(predictions)
    predictions = np.argmax(predictions, axis=2)
    return predictions


def decode_segmentation_masks(mask, colormap, n_classes):
    r = np.zeros_like(mask).astype(np.uint8)
    g = np.zeros_like(mask).astype(np.uint8)
    b = np.zeros_like(mask).astype(np.uint8)
    for l in range(0, n_classes):
        idx = mask == l
        r[idx] = colormap[l, 0]
        g[idx] = colormap[l, 1]
        b[idx] = colormap[l, 2]
    rgb = np.stack([r, g, b], axis=2)
    return rgb


def get_overlay(image, colored_mask):
    image = tf.keras.preprocessing.image.array_to_img(image)
    image = np.array(image).astype(np.uint8)
    overlay = cv2.addWeighted(image, 0.35, colored_mask, 0.65, 0)
    return overlay


def plot_samples_matplotlib(display_list, figsize=(5, 3)):
    _, axes = plt.subplots(nrows=1, ncols=len(display_list), figsize=figsize)
    for i in range(len(display_list)):
        if display_list[i].shape[-1] == 3:
            axes[i].imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        else:
            axes[i].imshow(display_list[i])
    plt.show()


def plot_predictions(images_list, colormap, model):
    for image_file in images_list:
        image_tensor = read_image(image_file)
        prediction_mask = infer(image_tensor=image_tensor, model=model)
        prediction_colormap = decode_segmentation_masks(prediction_mask, colormap, 20)
        overlay = get_overlay(image_tensor, prediction_colormap)
        plot_samples_matplotlib(
            [image_tensor, overlay, prediction_colormap], figsize=(18, 14)
        )

In [2]:
# # 딥랩v3 토치 파일들
# import torch
# from torch.utils.mobile_optimizer import optimize_for_mobile

# model = torch.hub.load('pytorch/vision:v0.11.0', 'deeplabv3_resnet50', pretrained=True)
# model.eval()

# scripted_module = torch.jit.script(model)
# optimized_scripted_module = optimize_for_mobile(scripted_module)

# # Export full jit version model (not compatible with lite interpreter)
# scripted_module.save("/content/drive/MyDrive/deeplabv3_scripted.pt")
# # Export lite interpreter version model (compatible with lite interpreter)
# scripted_module._save_for_lite_interpreter("/content/drive/MyDrive/deeplabv3_scripted.ptl")
# # using optimized lite interpreter model makes inference about 60% faster than the non-optimized lite interpreter model, which is about 6% faster than the non-optimized full jit model
# optimized_scripted_module._save_for_lite_interpreter("/content/drive/MyDrive/deeplabv3_scripted_optimized.ptl")

Downloading: "https://github.com/pytorch/vision/zipball/v0.11.0" to /root/.cache/torch/hub/v0.11.0.zip
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth


  0%|          | 0.00/161M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/torch/jit/_recursive.py:256: UserWarning: 'aux_classifier' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "
